<a href="https://colab.research.google.com/github/m-bashari-m/vehicle-color-recognition/blob/main/src/3_xyz_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py

--2022-06-14 19:11:36--  https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11733 (11K) [text/plain]
Saving to: ‘utils.py’

utils.py            100%[===================>]  11.46K  --.-KB/s    in 0s      

2022-06-14 19:11:37 (63.8 MB/s) - ‘utils.py’ saved [11733/11733]



In [2]:
from google.colab import drive
drive.mount('./drive')

Mounted at ./drive


In [3]:
import pandas as pd
import numpy as np
import os

import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub

!pip install -q tensorflow-io
import tensorflow_io as tfio

from utils import ModelCreator, get_train_val_ds, get_class_weight

     |████████████████████████████████| 25.9 MB 1.5 MB/s 


In [4]:
BATCH_SIZE = 32
IMG_SIZE = (256, 256)
AUTOTUNE = tf.data.AUTOTUNE
N_CLASSES = 16
HUB_URL = 'https://tfhub.dev/google/bit/m-r50x1/1'

In [5]:
dataset_dir = os.path.join('drive', 'MyDrive', 'cars')
train_dir = os.path.join(dataset_dir, 'train')
val_dir = os.path.join(dataset_dir, 'val')

In [6]:
train_ds, _ = get_train_val_ds(train_dir, val_dir, batch_size=BATCH_SIZE, img_size=IMG_SIZE)

Found 16580 files belonging to 16 classes.
Found 3508 files belonging to 16 classes.


In [7]:
classes, class_weight = get_class_weight()

In [8]:
train_ds = (
    train_ds.
    map(lambda img, lbl: (img/255., lbl), num_parallel_calls=AUTOTUNE).
    map(lambda img, lbl: (tfio.experimental.color.rgb_to_xyz(img), lbl), num_parallel_calls=AUTOTUNE).
    prefetch(AUTOTUNE))

In [9]:
xyz_creator = ModelCreator(HUB_URL, 'XYZ-model')
xyz_model = xyz_creator.make_model(img_size=IMG_SIZE)

Model: "XYZ-model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 2048)              23500352  
                                                                 
 dense (Dense)               (None, 16)                32784     
                                                                 
Total params: 23,533,136
Trainable params: 32,784
Non-trainable params: 23,500,352
_________________________________________________________________


In [10]:
callbacks = xyz_creator.get_callbacks()

In [11]:
history = xyz_model.fit(train_ds,
                        callbacks=callbacks,
                        epochs=25,
                        class_weight=class_weight)

Epoch 1/25
519/519 [==============================] - 2249s 4s/step - loss: 149.8451 - auc: 0.5486 - accuracy: 0.6311
Epoch 2/25
519/519 [==============================] - 183s 349ms/step - loss: 88.8327 - auc: 0.6507 - accuracy: 0.7224
Epoch 3/25
519/519 [==============================] - 187s 357ms/step - loss: 79.8329 - auc: 0.6778 - accuracy: 0.7495
Epoch 4/25
519/519 [==============================] - 184s 352ms/step - loss: 67.0500 - auc: 0.7218 - accuracy: 0.7811
Epoch 5/25
519/519 [==============================] - 181s 346ms/step - loss: 53.5126 - auc: 0.7498 - accuracy: 0.7984
Epoch 6/25
519/519 [==============================] - 184s 352ms/step - loss: 48.4253 - auc: 0.7664 - accuracy: 0.8119
Epoch 7/25
519/519 [==============================] - 180s 344ms/step - loss: 40.3471 - auc: 0.7937 - accuracy: 0.8296
Epoch 8/25
519/519 [==============================] - 184s 352ms/step - loss: 34.6213 - auc: 0.8127 - accuracy: 0.8429
Epoch 9/25
519/519 [=============================

In [12]:
!cp ./logs/checkpoints/XYZ-model-10.h5 ./drive/MyDrive/checkpoints/xyz-model-10.h5
!cp ./logs/checkpoints/XYZ-model-15.h5 ./drive/MyDrive/checkpoints/xyz-model-15.h5
!cp ./logs/checkpoints/XYZ-model-20.h5 ./drive/MyDrive/checkpoints/xyz-model-20.h5
!cp ./logs/checkpoints/XYZ-model-25.h5 ./drive/MyDrive/checkpoints/xyz-model-25.h5

In [14]:
history = xyz_model.fit(train_ds,
                        callbacks=callbacks,
                        epochs=30,
                        initial_epoch=25,
                        class_weight=class_weight)

Epoch 26/30
519/519 [==============================] - 187s 355ms/step - loss: 2.8084 - auc: 0.9810 - accuracy: 0.9655
Epoch 27/30
519/519 [==============================] - 181s 346ms/step - loss: 2.4569 - auc: 0.9837 - accuracy: 0.9669
Epoch 28/30
519/519 [==============================] - 186s 355ms/step - loss: 2.1919 - auc: 0.9850 - accuracy: 0.9707
Epoch 29/30
519/519 [==============================] - 184s 352ms/step - loss: 1.9954 - auc: 0.9872 - accuracy: 0.9701
Epoch 30/30
519/519 [==============================] - 181s 345ms/step - loss: 1.7814 - auc: 0.9888 - accuracy: 0.9728


In [15]:
!cp ./logs/checkpoints/XYZ-model-30.h5 ./drive/MyDrive/checkpoints/xyz-model-30.h5